In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.metrics import root_mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
from skopt import BayesSearchCV
from skopt.space import Integer, Real
import pickle
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('../dataset/data.csv')
data = df.select_dtypes(include=['float64', 'int64'])
targets = ["Turbidity", "DO", "fChl"]
data.columns

Index(['Turbidity', 'DO', 'fChl', 'Discharge', 'Height', 'WT', 'B1', 'B2',
       'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12', 'WVP',
       'MNDWI', 'GNDVI', 'SDDI', 'NDTI', 'BR', 'NDWI', 'NDPI', 'NDCI',
       '2BDA_Chl', 'RR'],
      dtype='object')

In [3]:
def xgb(data, target):
    X = data.drop(target, axis=1)
    y = data[target]

    model = XGBRegressor(
        random_state=42,
        n_jobs=-1,
        objective='reg:squarederror'
    )

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model.fit(X_train, y_train)

    os.makedirs("../models/xgb/split", exist_ok=True)
    save_path = f"../models/xgb/split/{target}.pkl"
    with open(save_path, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved to {save_path}")

    y_pred = model.predict(X_test)
    mae = np.around(mean_absolute_error(y_test, y_pred), 2)
    rmse = np.around(root_mean_squared_error(y_test, y_pred), 2)
    r2 = np.around(r2_score(y_test, y_pred) * 100, 2)
    mbe = np.around(np.mean(y_pred - y_test), 2)

    print(f"Performance for {target} (Split):")
    print(f"MAE  = {mae}")
    print(f"RMSE = {rmse}")
    print(f"R²   = {r2} %")
    print(f"MBE  = {mbe}")

In [4]:
for target in targets:
    xgb(
        data.drop([col for col in targets if col != target], axis=1),
        target
    )
    print("="*40)

Model saved to ../models/xgb/split/Turbidity.pkl
Performance for Turbidity (Split):
MAE  = 9.61
RMSE = 12.41
R²   = 84.66 %
MBE  = 3.39
Model saved to ../models/xgb/split/DO.pkl
Performance for DO (Split):
MAE  = 0.37
RMSE = 0.49
R²   = 95.2 %
MBE  = -0.01
Model saved to ../models/xgb/split/fChl.pkl
Performance for fChl (Split):
MAE  = 1.62
RMSE = 2.07
R²   = 81.52 %
MBE  = -0.21


In [5]:
def cvxgb(data, target):
    X = data.drop(target, axis=1)
    y = data[target]

    model = XGBRegressor(
        random_state=42,
        n_jobs=-1,
        objective='reg:squarederror'
    )

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    y_pred = cross_val_predict(model, X, y, cv=kf, n_jobs=-1)
    model.fit(X, y)
    
    os.makedirs("../models/xgb/cross", exist_ok=True)
    save_path = f"../models/xgb/cross/{target}.pkl"
    with open(save_path, 'wb') as file:
        pickle.dump(model, file)
    print(f"Model saved to {save_path}")

    mae = np.around(mean_absolute_error(y, y_pred), 2)
    rmse = np.around(root_mean_squared_error(y, y_pred), 2)
    r2 = np.around(r2_score(y, y_pred) * 100, 2)
    mbe = np.around(np.mean(y_pred - y), 2)

    print(f"Performance for {target} (5-Fold CV):")
    print(f"MAE  = {mae}")
    print(f"RMSE = {rmse}")
    print(f"R²   = {r2} %")
    print(f"MBE  = {mbe}")

In [6]:
for target in targets:
    cvxgb(
        data.drop([col for col in targets if col != target], axis=1),
        target
    )
    print("="*40)

Model saved to ../models/xgb/cross/Turbidity.pkl
Performance for Turbidity (5-Fold CV):
MAE  = 7.84
RMSE = 11.06
R²   = 90.13 %
MBE  = 1.1
Model saved to ../models/xgb/cross/DO.pkl
Performance for DO (5-Fold CV):
MAE  = 0.48
RMSE = 0.69
R²   = 88.73 %
MBE  = 0.06
Model saved to ../models/xgb/cross/fChl.pkl
Performance for fChl (5-Fold CV):
MAE  = 1.74
RMSE = 2.29
R²   = 75.48 %
MBE  = -0.14
